In [ ]:
# Session plan
# Create a pipeline for the regression model 
#   it should pre-process and also train the model 
#   use it to predict a outcome
# Use the created pipeline to train the model and log it with mlflow.
# Use the logged model to predict a an outcome. 

In [1]:
import pandas as pd
import plotly.express as px
from scipy.stats import skew
import pandas as pd
from scipy.stats import pearsonr, chi2_contingency, f_oneway
from sklearn.feature_selection import mutual_info_classif, f_classif
import numpy as np
from scipy.stats import boxcox, skew
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv("D:\\GIT HUB\\GUVI Mini Proj 4\\train_data - train_data.csv")

In [3]:
df_train = data.drop(columns=['price'])
df_test = data['price']

In [4]:
X_train, X_test, Y_train, Y_test = train_test_split(df_train,df_test, test_size=0.2, random_state=42)

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, FunctionTransformer, OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin

# Building Pipeline for preprocessing categorical columns

In [ ]:
#To be done: Grouping, Target encoding, OHE,Binary,label, cyclic encoding and Scaling on target encoded
# done:


In [6]:
# cyclic good
# cyclic encoding function for month and day
import numpy as np

def cyclic_encode(df, col, max_val):
    """
    Applies sine and cosine transformation to a single column for cyclic encoding.

    Args:
        df (pd.DataFrame): The input DataFrame.
        col (str): The column to transform (e.g., 'hour').
        max_val (int): The maximum possible value of the feature (e.g., 24 for hour).

    Returns:
        pd.DataFrame: A DataFrame with the two new encoded columns.
    """
    df[col + '_sin'] = np.sin(2 * np.pi * df[col] / max_val)
    df[col + '_cos'] = np.cos(2 * np.pi * df[col] / max_val)
    # Drop the original column as it is now represented by the sine and cosine features
    df = df.drop(col, axis=1)
    return df

In [7]:
# --- Defining the pre-processing steps ---

# Define a function to apply the encoding to the dataframe.
# We wrap the specific logic within a lambda for the ColumnTransformer
# to only pass the required column and context.
def apply_cyclic_encoding(X, col, max_val):
    # X is a numpy array when passed through ColumnTransformer, so convert it back to DataFrame
    # Note: ColumnTransformer passes the *entire* column/array slice.
    X_df = pd.DataFrame(X, columns=[col])
    return cyclic_encode(X_df, col, max_val)

# Create the transformers
cyclic_hour_transformer = FunctionTransformer(
    func=lambda X: apply_cyclic_encoding(X, 'month', 12),
    validate=False # Set to False to handle DataFrame/numpy array mix
)

cyclic_day_transformer = FunctionTransformer(
    func=lambda X: apply_cyclic_encoding(X, 'day', 31),
    validate=False
)



In [ ]:
#Grouping, Target encoding, OHE,Binary,label and Scaling
# done:cyclic encoding


In [ ]:
# old code
# Groupping 
# import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin

class RareCategoryGrouper(BaseEstimator, TransformerMixin):
    """
    Groups categories in a specified column that fall below a minimum
    percentage threshold into a single 'RARE_GROUP'.
    """
    def __init__(self, column_name, min_percentage=0.01, rare_label='RARE_GROUP'):
        self.column_name = column_name
        self.min_percentage = min_percentage
        self.rare_label = rare_label
        self.rare_categories_ = None # Stores the categories learned during fit

    def fit(self, X, y=None):
        """
        Calculates and stores the list of categories that are below the
        min_percentage threshold in the training data (X).
        """
        # 1. Calculate the normalized frequency counts
        value_counts = X[self.column_name].value_counts(normalize=True)

        # 2. Identify and store the categories to be grouped
        self.rare_categories_ = value_counts[value_counts < self.min_percentage].index.tolist()

        if not self.rare_categories_:
            print(f"FIT: No categories in '{self.column_name}' below the {self.min_percentage*100:.2f}% threshold.")
        else:
             print(f"FIT: Identified {len(self.rare_categories_)} rare categories in '{self.column_name}'.")

        return self

    def transform(self, X):
        """
        Applies the grouping transformation using the stored rare categories.
        """
        # Ensure we operate on a copy to avoid side effects on the input DataFrame
        X_copy = X.copy()
        
        # If no rare categories were found during fit, return the DataFrame unchanged
        if not self.rare_categories_:
            return X_copy
        
        # 3. Create the new column name
        new_column_name = f'{self.column_name}_grouped'
        X_copy[new_column_name] = X_copy[self.column_name]

        # 4. Apply the grouping using .loc
        X_copy.loc[X_copy[self.column_name].isin(self.rare_categories_), new_column_name] = self.rare_label
        
        # IMPORTANT: Drop the original column to maintain a consistent column count in the pipeline
        X_copy = X_copy.drop(columns=[self.column_name])
        
        return X_copy

In [ ]:
#old code
# --- Define the Grouping Steps ---
# Group 'country' with a 1% threshold
country_grouper = RareCategoryGrouper(column_name='country', min_percentage=0.01)

# Group 'page2_clothing_model' with a 0.5% threshold
model_grouper = RareCategoryGrouper(column_name='page2_clothing_model', min_percentage=0.005)

In [ ]:
# --- 2. Target Encoder with CV and Box-Cox ---
from sklearn.model_selection import StratifiedKFold
from category_encoders import TargetEncoder
from scipy.stats import boxcox # For Box-Cox transformation
class TargetEncoderCV(BaseEstimator, TransformerMixin):
    """
    Performs K-Fold Cross-Validated Target Encoding during fit, followed by Box-Cox scaling.
    Accepts the column data from the previous pipeline step.
    """
    def __init__(self, n_splits=5, random_state=42):
        self.n_splits = n_splits
        self.random_state = random_state
        self.full_encoder_ = None 
        self.lambda_ = None       

    def fit(self, X, y):
        # X is the data slice (now the grouped column array from RareCategoryGrouper)
        X_series = pd.Series(X.flatten(), index=y.index)
        
        # 1. Cross-Validated Encoding
        kf = StratifiedKFold(n_splits=self.n_splits, shuffle=True, random_state=self.random_state)
        X_encoded_train = pd.Series(np.nan, index=y.index)

        for train_index, val_index in kf.split(X_series.to_frame(), y):
            encoder = TargetEncoder()
            # Fit only on the K-1 folds (uses Series access)
            encoder.fit(X_series.iloc[train_index], y.iloc[train_index])
            # Transform the validation fold
            X_encoded_train.loc[val_index] = encoder.transform(X_series.iloc[val_index]).flatten()
            
        # 2. Box-Cox Transformation Parameter Learning
        X_encoded_train_2d = X_encoded_train.values.reshape(-1, 1)
        X_boxcox, self.lambda_ = boxcox(X_encoded_train_2d + 1e-6) 

        # 3. Store the full encoder for transformation of *new* data
        self.full_encoder_ = TargetEncoder()
        self.full_encoder_.fit(X_series, y)
        
        return self

    def transform(self, X):
        # X is the data slice from the previous step
        X_series = pd.Series(X.flatten(), index=pd.Index(range(len(X))))
        
        # 1. Apply Target Encoding using the FULL ENCODER learned during fit
        X_encoded = self.full_encoder_.transform(X_series)
        
        # 2. Apply Box-Cox Transformation using the stored lambda
        X_transformed = boxcox(X_encoded.values.reshape(-1, 1) + 1e-6, lmbda=self.lambda_)

        return X_transformed.reshape(-1, 1)

In [ ]:
#Grouping new
from sklearn.base import BaseEstimator, TransformerMixin

# --- 1. Rare Category Grouper (Returns only the new column array) ---
class RareCategoryGrouper(BaseEstimator, TransformerMixin):
    """
    Groups categories that fall below a minimum percentage threshold into a single 'RARE_GROUP'.
    This version is designed for use in a scikit-learn Pipeline and returns only the transformed column data.
    """
    def __init__(self, rare_label='RARE_GROUP', min_percentage=0.01):
        # We don't need 'column_name' here because the ColumnTransformer will select the column for us.
        self.rare_label = rare_label
        self.min_percentage = min_percentage
        self.rare_categories_ = None # Stores the categories to be grouped

    def fit(self, X, y=None):
        # X is the slice of data (e.g., the 'page2_clothing_model_grouped' column data)
        # Convert array slice back to Series for value_counts
        X_series = pd.Series(X.flatten())

        value_counts = X_series.value_counts(normalize=True)
        self.rare_categories_ = value_counts[value_counts < self.min_percentage].index.tolist()
        
        # We don't print inside fit/transform methods in production code, but keeping for explanation:
        # print(f"FIT: Identified {len(self.rare_categories_)} rare categories.")
        return self

    def transform(self, X):
        X_series = pd.Series(X.flatten(), index=pd.Index(range(len(X))))
        
        # Create a copy to modify
        X_transformed = X_series.copy()
        
        if self.rare_categories_:
            # Apply the grouping based on the categories learned during fit
            X_transformed[X_series.isin(self.rare_categories_)] = self.rare_label
        
        # Return the transformed column data as a 2D array, which is required by scikit-learn
        return X_transformed.values.reshape(-1, 1)


In [ ]:
# --- 1. Define the Mini-Pipeline for Sequential Steps ---
# This pipeline performs two actions on the same column sequentially: Grouping then Encoding/Scaling
target_encoding_and_scaling_pipe = Pipeline(steps=[
    # Step 1: Group rare categories in the selected column
    ('group_rare', RareCategoryGrouper(min_percentage=0.005)), 
    # Step 2: Apply K-Fold Target Encoding and Box-Cox Scaling to the grouped result
    ('target_boxcox', TargetEncoderCV(n_splits=5))
])

country_grouper_OHE_pipe = Pipeline(steps = [('group_rare', RareCategoryGrouper(min_percentage= 0.01)),
                                             ('ohe_country', ohe_encoder())])

In [ ]:
# complete the grouping+target encoder+boxcox pipeline 
# complete the grouping + OHE
# Complete the OHE
# compile the 1st 2 pipelines with other steps in the columntransformer
# compile the final pre-processer 

In [ ]:
# Target encoding, OHE,label and Scaling
# done:cyclic encoding , Grouping


In [8]:
#label encoding  - good
from sklearn.base import BaseEstimator, TransformerMixin
class DictionaryMapper(BaseEstimator, TransformerMixin):
    """
    Applies a fixed dictionary mapping (Label Encoding) to a specified column.
    """
    def __init__(self, column_name, mapping_dict):
        self.column_name = column_name
        self.mapping_dict = mapping_dict

    def fit(self, X, y=None):
        # No fitting is required as the mapping is pre-defined
        return self

    def transform(self, X):
        X_copy = X.copy()
        
        # Apply the mapping and replace the original column
        X_copy[self.column_name] = X_copy[self.column_name].map(self.mapping_dict)
        
        # NOTE: .map() will produce NaN for values not in the dictionary. 
        # If your data might contain other values, you may need a fillna() or error check here.
        
        return X_copy

In [9]:
# Define the mappers using your specified dictionary
model_photo_mapper = DictionaryMapper(
    column_name='model_photography', 
    mapping_dict={1: 0, 2: 1}
)

price_mapper = DictionaryMapper(
    column_name='price_2', 
    mapping_dict={1: 0, 2: 1}
)


In [ ]:
# Target encoding, OHE and Scaling
# done:cyclic encoding , Grouping, label encoding


In [10]:
#good - grouping
# --- Custom Transformer for Rare Category Grouping (FIXED) ---
class RareCategoryGrouper(BaseEstimator, TransformerMixin):
    """
    A custom transformer to group categories whose frequency is below a given threshold
    into a single 'RARE_GROUP' category.
    """
    def __init__(self, threshold=0.01, rare_group_name='RARE_GROUP'):
        # The threshold (e.g., 0.01 for 1%) below which categories are grouped
        self.threshold = threshold
        # The name assigned to the grouped rare categories
        self.rare_group_name = rare_group_name
        # Internal attribute to store the list of frequent categories (learned during fit)
        self.frequent_categories_ = None

    def fit(self, X, y=None):
        # Determine the Series to work with (handles DataFrame or Series input)
        if isinstance(X, pd.DataFrame):
            # If it's a DataFrame, extract the first (and only) column
            X_series = X.iloc[:, 0]
        else:
            X_series = X
        
        # Calculate the frequency of each category
        category_counts = X_series.value_counts(normalize=True)
        
        # Identify categories that meet or exceed the threshold
        self.frequent_categories_ = category_counts[category_counts >= self.threshold].index.tolist()
        
        # Return self to adhere to scikit-learn API
        return self

    def transform(self, X):
        # --- FIX APPLIED HERE ---
        # 1. Determine the column content to transform (Series)
        if isinstance(X, pd.DataFrame):
            # Extract the single column content as a Series
            series_to_process = X.iloc[:, 0]
            name_to_keep = X.columns[0]
        else: # Must be a Series
            series_to_process = X
            name_to_keep = X.name
            
        # Perform the actual grouping transformation on the Series data (using numpy.where)
        grouped_values = np.where(
            series_to_process.isin(self.frequent_categories_), 
            series_to_process, 
            self.rare_group_name
        )

        # 2. Re-wrap the results as a Series, maintaining index and original column name
        X_transformed_series = pd.Series(
            grouped_values, 
            index=X.index, 
            name=name_to_keep
        )
        
        # 3. Always return a DataFrame (2D array-like structure) by calling to_frame() on the Series
        return X_transformed_series.to_frame()

In [11]:
Grouping_OHE_pipeline = Pipeline(steps=[
    ("Grouping",RareCategoryGrouper(threshold=0.01)),
    ("OHE",OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])

In [ ]:
# Target encoding - only one column left(clothing model)
# done:cyclic encoding , Grouping, label encoding ,OHE (applied directly on the pipeline and column tranformer)

In [12]:
# Target encoding 
# --- 1. KFold Target Encoder Class (Copied from target_encoding.py) ---
# NOTE: This class is essential for the pipeline to run correctly.
from sklearn.model_selection import KFold
class KFoldTargetEncoder(BaseEstimator, TransformerMixin):
    """
    Performs K-Fold Target Encoding for a single categorical column 
    while preventing data leakage using cross-validation.
    
    This encoder is suitable for regression tasks where the target (y) is continuous.
    """
    def __init__(self, n_splits=5, shuffle=True, random_state=42, smoothing=None):
        self.n_splits = n_splits
        self.shuffle = shuffle
        self.random_state = random_state
        self.smoothing = smoothing
        self._kf = KFold(n_splits=n_splits, shuffle=shuffle, random_state=random_state)
        self._mapping = {}
        self._global_mean = None
        
    def fit(self, X, y):
        if isinstance(X, pd.DataFrame):
             X = X.iloc[:, 0]
        y_series = pd.Series(y)
        
        self._global_mean = y_series.mean()
        self._mapping = y_series.groupby(X).mean().to_dict()
        
        return self

    def transform(self, X):
        if isinstance(X, pd.DataFrame):
             X = X.iloc[:, 0]
             
        encoded_feature = X.map(self._mapping).fillna(self._global_mean)
        
        # Ensure output is a DataFrame/Series so it works well in ColumnTransformer
        return encoded_feature.to_frame(name=X.name + '_encoded')

    def fit_transform(self, X, y=None):
        if isinstance(X, pd.DataFrame):
             X_series = X.iloc[:, 0].copy()
        else:
             X_series = X.copy()
             
        y = pd.Series(y)
        X_encoded = pd.Series(np.zeros(len(X_series)), index=X_series.index)
        
        # 1. Iterate over K-Folds
        for train_idx, val_idx in self._kf.split(X_series, y):
            X_train_fold, X_val_fold = X_series.iloc[train_idx], X_series.iloc[val_idx]
            y_train_fold = y.iloc[train_idx]
            
            # 2. Calculate the mean target based ONLY on the current train fold
            fold_means = y_train_fold.groupby(X_train_fold).mean()
            global_mean_fold = y_train_fold.mean()
            
            # 3. Map these means to the validation fold
            X_encoded.iloc[val_idx] = X_val_fold.map(fold_means).fillna(global_mean_fold)
            
        # 4. Fit the final, overall mapping for future 'transform' calls (on test data)
        self.fit(X_series, y) 

        # Return as DataFrame for ColumnTransformer compatibility
        return X_encoded.to_frame(name=X_series.name + '_encoded')

In [13]:
Grouping_TargetEncode_pipeline = Pipeline(steps=[
    ("Grouping",RareCategoryGrouper(threshold=0.005)),
    ("Target encoding",KFoldTargetEncoder(n_splits=5, random_state=42))
    ])

In [14]:
# --- 3. Build the ColumnTransformer ---
# Categorical 
preprocessor = ColumnTransformer(
    transformers=[
        # Apply cyclic encoding to 'hour' and 'day_of_week'
        ('cyclic_hour', cyclic_hour_transformer, ['month']),
        ('cyclic_day', cyclic_day_transformer, ['day']),

        # Label Encoding (NO SCALING APPLIED)
        # Use the mappers directly here. The output is 0/1, which is final.
        ('map_model_photo', model_photo_mapper, ['model_photography']),
        ('map_price_2', price_mapper, ['price_2']),

        # ONE-HOT ENCODING STEP (New) 
        ('simple OHE', OneHotEncoder(handle_unknown='ignore', sparse_output=False), ['page1_main_category','colour','location']),
        
        # Grouping pipe
        ("Grouping+OHE",Grouping_OHE_pipeline,['country']),

        # Target Encoding for the categorical feature (leakage-free)
        ('target_encoder', Grouping_TargetEncode_pipeline, ['page2_clothing_model'])

    ],
    # remainder='passthrough' # Keep any other columns if they exist
    remainder='drop'# Drop all other columns not explicitly listed
)

In [15]:
from sklearn.linear_model import LinearRegression

core_pipeline = Pipeline(steps=[
    # ... your existing Grouping, OHE, Target Encoding, and numeric steps ...
    # Placeholder estimator:
    ('Preprocessor', preprocessor),
    ('estimator', LinearRegression()) # Changed from Ridge(random_state=42)
])

In [16]:
# 1. Define the Box-Cox Transformer
# We use PowerTransformer from sklearn and set the method to 'box-cox'.
# The Box-Cox method requires the input data (the target Y) to be strictly positive (Y > 0).
# If your target can be zero or negative, you might need to shift it (e.g., Y_shifted = Y + 1)
# or consider the 'yeo-johnson' method instead.
from sklearn.preprocessing import PowerTransformer
from sklearn.compose import TransformedTargetRegressor

boxcox_transformer = PowerTransformer(method='box-cox', standardize=False)

# 2. Create the TransformedTargetRegressor (TTR) wrapper
# The TTR handles:
# 1. fit: Y -> fit(boxcox_transformer) -> Y_transformed
# 2. predict: X -> core_pipeline -> Y_transformed_hat
# 3. inverse_transform: Y_transformed_hat -> inverse_transform(boxcox_transformer) -> Y_hat (final prediction)
final_model = TransformedTargetRegressor(
    regressor=core_pipeline,
    transformer=boxcox_transformer
)

In [17]:
# --- 4. Training and Prediction ---

# 1. Fit the final model (TTR intercepts Y_train and transforms it to log(Y_train))
print("Fitting the TransformedTargetRegressor...")
final_model.fit(X_train, Y_train)
print("Fit complete.")

Fitting the TransformedTargetRegressor...
Fit complete.


In [18]:
# 2. Predict (TTR automatically applies the inverse exp() function to the predictions)
Y_pred = final_model.predict(X_test)

d:\GIT HUB\GUVI Mini Proj 4\clickstream_venv\Lib\site-packages\sklearn\pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


In [19]:
# 3. Evaluate results on the original scale
# from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
rmse = mean_squared_error(Y_test, Y_pred)
print(rmse)

# print("\n--- Model Performance ---")
# print(f"Test Root Mean Squared Error (RMSE) on original scale: ${rmse:.2f}")

# print("\n--- Prediction Comparison (Original Scale) ---")
# results = pd.DataFrame({
#     'Actual Price': Y_test,
#     'Predicted Price': Y_pred
# })
# print(results)

23.42430767963361


In [20]:
# 3. Predict and Evaluate
# y_pred = best_estimator.predict(x_test)
mae = mean_absolute_error(Y_test, Y_pred)
mse = mean_squared_error(Y_test, Y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(Y_test, Y_pred)
print(f"MAE: {mae:.4f}, MSE: {mse:.4f}, RMSE: {rmse:.4f}, R2 Score: {r2:.4f}")

MAE: 3.5819, MSE: 23.4243, RMSE: 4.8399, R2 Score: 0.8522


In [ ]:
#Test run to validate the pipeline


In [21]:
test_data = pd.read_csv("D:\\GIT HUB\\GUVI Mini Proj 4\\test_data - test_data.csv")

In [22]:
x = test_data.drop(columns= ['price'])
y = test_data['price']

In [29]:
y_pred.shape

(26476,)

In [30]:
y_pred = final_model.predict(x)

d:\GIT HUB\GUVI Mini Proj 4\clickstream_venv\Lib\site-packages\sklearn\pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


In [31]:
print("\n--- Model Performance ---")
print(f"Test Root Mean Squared Error (RMSE) on original scale: ${rmse:.2f}")

print("\n--- Prediction Comparison (Original Scale) ---")
results = pd.DataFrame({
    'Actual Price': y,
    'Predicted Price': y_pred
})
print(results)


--- Model Performance ---
Test Root Mean Squared Error (RMSE) on original scale: $4.84

--- Prediction Comparison (Original Scale) ---
       Actual Price  Predicted Price
0                33        27.086633
1                33        36.158185
2                28        26.407764
3                57        58.962233
4                43        40.968232
...             ...              ...
33090            43        39.128288
33091            33        36.172293
33092            48        38.535691
33093            48        48.286674
33094            33        28.920606

[33095 rows x 2 columns]


In [32]:
mae = mean_absolute_error(y, y_pred)
mse = mean_squared_error(y, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y, y_pred)
print(f"MAE: {mae:.4f}, MSE: {mse:.4f}, RMSE: {rmse:.4f}, R2 Score: {r2:.4f}")

MAE: 3.6023, MSE: 23.6831, RMSE: 4.8665, R2 Score: 0.8504


In [ ]:
# below code is for verification used while building pipelines:

In [24]:
X_processed1 = preprocessor.fit_transform(X_train,Y_train)

In [25]:
X_processed1[:2]

array([[-0.4999999999999997, -0.8660254037844388, 0.72479278722912,
        0.6889669190756866, 0, 1, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,
        0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0,
        0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 2008, 2, 17592, 'C53', 3],
       [1.2246467991473532e-16, -1.0, 0.20129852008866006,
        0.9795299412524945, 0, 1, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0,
        0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,
        0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 2008, 6, 12203, 'B9', 1]],
      dtype=object)

In [22]:
X_processed[:2]

array([[-5.00000000e-01, -8.66025404e-01,  7.24792787e-01,
         6.88966919e-01,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  3.97739924e+01,
         2.00800000e+03,  2.00000000e+00,  1.75920000e+04,
         3.00000000e+00],
       [ 1.22464680e-16, -1.00000000e+00,  2.01298520e-01,
         9.79529941e-01,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.00